In [67]:
from urllib.parse import urlparse, parse_qs, urlencode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from secrets import token_bytes

def qs(query):
    return {k:v[0] for k,v in parse_qs(query).items()}

def profile_for(email):
    qs = {'email':email, 'uid':10, 'role':'user'}
    #return urlencode(qs, safe="").encode()
    _filter = "".join([chr(b) for b in range(256)]).replace("&", "").replace("=", "")
    return urlencode(qs, safe=_filter).encode()

def split16(string):
    start = 0
    end = 16
    while True:
        print("|{:<16}| {}".format(string[start:end], len(string[start:end])))
        if len(string[start:end]) == 0:
            print("")
            return
        start += 16
        end += 16
    
class ENC:
    key = token_bytes(AES.block_size)
    
    def encrypt(self, plaintext):
        aes_ecb = AES.new(self.key, AES.MODE_ECB)
        plaintext = pad(plaintext, AES.block_size)
        split16(plaintext.decode())
        return aes_ecb.encrypt(plaintext)

    def decrypt(self, cyphertext):
        aes_ecb = AES.new(self.key, AES.MODE_ECB)
        return unpad(aes_ecb.decrypt(cyphertext), 16)
    
profile_for("a@a.a&&role=admin")

b'email=a@a.a%26%26role%3Dadmin&uid=10&role=user'

In [68]:
enc = ENC()
_input = "&role=admin"
print(len(_input))
prof = profile_for("AAAAAAAAAA" + _input + _input + _input)
split16(prof.decode())
encrypted = enc.encrypt(prof)
split16(encrypted.hex())
decrypted = enc.decrypt(encrypted)
print(decrypted)
parse = qs(decrypted)
print(parse)

11
|email=AAAAAAAAAA| 16
|%26role%3Dadmin%| 16
|26role%3Dadmin%2| 16
|6role%3Dadmin&ui| 16
|d=10&role=user  | 14
|                | 0

|email=AAAAAAAAAA| 16
|%26role%3Dadmin%| 16
|26role%3Dadmin%2| 16
|6role%3Dadmin&ui| 16
|d=10&role=user| 16
|                | 0

|c8bfef78327ea65b| 16
|00136531c20471c2| 16
|1af75c959f24911f| 16
|d7c1bae62650d697| 16
|74e03253ab59ede4| 16
|1fbe939ba4f2d5a3| 16
|2364bf73051dfdb4| 16
|4648240ff392b0ee| 16
|287a6e51e7835c40| 16
|4d0ec2347c9ddcfd| 16
|                | 0

b'email=AAAAAAAAAA%26role%3Dadmin%26role%3Dadmin%26role%3Dadmin&uid=10&role=user'
{b'email': b'AAAAAAAAAA&role=admin&role=admin&role=admin', b'uid': b'10', b'role': b'user'}


In [29]:
block = "email=a@a.c&uid=".encode()
print("block: ", len(block))
digest = enc.encrypt(block).hex()[:32]
print(digest, len(digest))

block2 = "10000&role=admin".encode()
print("block2: ", len(block2))
digest = enc.encrypt(block2).hex()[:32]
print(digest, len(digest))

block:  16
|email=a@a.c&uid=| 16
|| 16

75089ae571852c1cf54e483c47bccc13 32
block2:  16
|10000&role=admin| 16
|| 16

cec55e0c93f42ebb8e77e1f0cec26405 32


In [72]:
enc.encrypt(profile_for("AAAAAAAAAA"))

for char in range(2):
    char = bytes([char])
    guess = b"AAAAAAAAAA" + char + b"&role=admin"
    print(guess, len(guess), len(guess) + len("email="))
    split16(enc.encrypt(profile_for(guess)).hex())

|email=AAAAAAAAAA| 16
|&uid=10&role=use| 16
|r| 16
|                | 0

b'AAAAAAAAAA\x00&role=admin' 22 28
|email=AAAAAAAAAA| 16
| %26role%3Dadmin| 16
|&uid=10&role=use| 16
|r| 16
|                | 0

|c8bfef78327ea65b| 16
|00136531c20471c2| 16
|c3102d688fad7353| 16
|1422a0216c1e0da4| 16
|b5a1043cfb26b243| 16
|eaac832edb55b5ce| 16
|931531619d2fab8b| 16
|d5c112293be829dc| 16
|                | 0

b'AAAAAAAAAA\x01&role=admin' 22 28
|email=AAAAAAAAAA| 16
|%26role%3Dadmin| 16
|&uid=10&role=use| 16
|r| 16
|                | 0

|c8bfef78327ea65b| 16
|00136531c20471c2| 16
|d437061cd60b35dc| 16
|c64d09d05c6b7039| 16
|b5a1043cfb26b243| 16
|eaac832edb55b5ce| 16
|931531619d2fab8b| 16
|d5c112293be829dc| 16
|                | 0

